In [2]:
from PIL import Image, ImageFilter
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2
import Func
import time
import os

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# intro-mnist  莫烦教程

In [ ]:
# 莫烦教程

from tensorflow.examples.tutorials.mnist import input_data  
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)  

def add_layer(inputs,in_size,out_size,activation_function = None):
    Weights = tf.Variable(tf.random_normal([in_size,out_size]),name = 'W')
    biases = tf.Variable(tf.zeros([1,out_size])+0.1,name = 'b')
    Wx_puls_b = tf.add(tf.matmul(inputs,Weights),biases)
    if activation_function is None:
        outputs = Wx_puls_b 
    else:
        outputs = activation_function(Wx_puls_b,)
    return outputs

def compute_accuracy(v_xs,v_ys):
    global prediction
    y_pre = sess.run(prediction,feed_dict={x:v_xs})
    correct_prediction = tf.equal(tf.argmax(y_pre,1),tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    result = sess.run(accuracy,feed_dict={x:v_xs,y_:v_ys})
    return result

#define placeholder for inputs to netword
x = tf.placeholder("float", [None, 784])  #28*28
y_ = tf.placeholder("float", [None,10])

#add ouyput layer
prediction = add_layer(x,784,10,activation_function = tf.nn.softmax)

#the error between prediction and real data 计算交叉熵:
#cross_entropy = -tf.reduce_sum(y_*tf.log(y))
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_*tf.log(prediction),
                                              reduction_indices=[1]))#loss
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(1001):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    if i%50 == 0:  
#         correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
#         accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
#         print ("step %d, training accuracy %g"%(i, sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})))  
          print(i,compute_accuracy(mnist.test.images,mnist.test.labels))
# saver = tf.train.Saver()   
# saver.save(sess, "model_data_intro/model.ckpt")  


# intro-mnist  官方教程

In [ ]:

# 官方教程

time_start=time.time()


from tensorflow.examples.tutorials.mnist import input_data  
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)  

#define placeholder for inputs to netword
x = tf.placeholder("float", [None, 784])  #28*28
y_ = tf.placeholder("float", [None,10])

W = tf.Variable(tf.zeros([784,10]),dtype = tf.float32,name = 'weights')
b = tf.Variable(tf.zeros([10]),dtype = tf.float32,name = 'biases')

y = tf.nn.softmax(tf.matmul(x,W) + b)

y_ = tf.placeholder("float", [None,10])
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)


sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(1001):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    if i%100 == 0:  
        correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        print ("step %d, training accuracy %g"%(i, sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})))  
saver = tf.train.Saver()   
saver.save(sess, "./model/intro/model.ckpt")  

time_end=time.time();#time.time()为1970.1.1到当前时间的毫秒数  
print ((time_end-time_start)/60,"mins")  

# expert-mnist 

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data  
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)  

time_start=time.time()
sess = tf.InteractiveSession()  
  
#定义一些函数：分配系数函数、分配偏置函数、卷积函数、pooling函数  
def weight_variable(shape):  
    initial = tf.truncated_normal(shape, stddev=0.1)#均值0标准方差0.1，剔除2倍标准方差之外的随机数据  
    return tf.Variable(initial)  
    
def bias_variable(shape):  
    initial = tf.constant(0.1, shape=shape)#统一值0.1  
    return tf.Variable(initial)  
  
def conv2d(x, W):  
  #待操作的数据x，模板W，tensor不同维度上的步长，强制与原tensor等大  
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')  
  
def max_pool_2x2(x):  
    #平面数据的pool模板2*2，平面数据滑动步长2*2（非重叠的pool）  
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')  
  
  
 #x是输入的图像，y_是对应的标签  
x = tf.placeholder(tf.float32, [None, 784])  
y_ = tf.placeholder(tf.float32, [None, 10])  
  
#第1层卷积层，Receptive Field 5＊5，单个batch生成32通道数据  
W_conv1 = weight_variable([5, 5, 1, 32])  
b_conv1 = bias_variable([32])  
  
#把图像向量还原成28＊28的图像  
x_image = tf.reshape(x, [-1,28,28,1])  
 
#第1个卷积层，使用了ReLU激活函数  
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)  
h_pool1 = max_pool_2x2(h_conv1)  
  
#第2层卷积层，Receptive Field 5＊5，单个batch 32通道生成64通道数据  
W_conv2 = weight_variable([5, 5, 32, 64])  
b_conv2 = bias_variable([64])  
  
#第2个卷积层  
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)  
h_pool2 = max_pool_2x2(h_conv2)  
  
#全链接层系数  
W_fc1 = weight_variable([7 * 7 * 64, 1024])  
b_fc1 = bias_variable([1024])  
  
#全链接层：把64通道数据展开方便全链接  
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])  
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)  
  
#全链层神经元使用dropout防止过拟合  
keep_prob = tf.placeholder("float")  
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)  
  
#softmax层系数  
W_fc2 = weight_variable([1024, 10])  
b_fc2 = bias_variable([10])  
  
#softmax层  
y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)  
  
#交叉熵和训练构型：AdamOptimizer适合这种求和的误差项  
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))  
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)  
  
#验证步骤的构型  
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))  
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))  
  
#初始化  
sess.run(tf.initialize_all_variables())  
  
#开始训练  
for i in range(1000):  
    batch = mnist.train.next_batch(50)  
    if i%100 == 0:  
        #验证的时候dropout=1.0，训练时=0.5  
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y_: batch[1], keep_prob: 1.0})  
        print ("step %d, training accuracy %g"%(i, train_accuracy))  
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})  
  
#验证最终的准确率  
print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}) )  

saver = tf.train.Saver()   
saver.save(sess, "./model/expert/model.ckpt")  

time_end=time.time();#time.time()为1970.1.1到当前时间的毫秒数  
print ((time_end-time_start)/60,"mins")  

# 算式图片切割

In [ ]:

#将算式中的多个字符分别切割保存,保存路径为 file_dir+"/imgi.png"
def CutImage(img,file_dir):
    img = np.asarray(img)
    L=0
    R=0
    lock = 0
    cnt=0
     #寻找左右边界
    for i in range(len(img[0])):
        if Func.Judgecolzero(img,i)==1 and lock==0:
            L=i
            lock=1
        if Func.Judgecolzero(img,i)==0 and lock==1:
            R=i
            lock=0
            img1 = im.crop((L-15,0,R+15,len(img)-1))
            img2 = Func.Thumbnail(img1.crop(Func.JudgeEdge(img1)),28)
            img2.save(file_dir+"img"+str(cnt)+".png")
            L=0
            R=0
            cnt=cnt+1
    return cnt
        
file_name='images_data/s.png'#导入自己的图片地址

im = Image.open(file_name).convert('1')

cnt = CutImage(im,"images_data/img/")#将算式中的多个字符分别切割保存,保存路径为 file_dir+"/imgi.png"
print("ok",cnt)

# 成绩表格图片切割校正

In [ ]:
def Adjust_img(file_name):
    img = cv2.imread(file_name)
    result = img.copy()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    #ret,thresh = cv2.threshold(gray,190,255,cv2.THRESH_BINARY)
    ret,thresh = cv2.threshold(gray,190,255,cv2.THRESH_BINARY_INV) #反色读入

    #binary, contours, hierarchy = cv2.findContours(eroded,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    binary, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

    x, y, w, h = cv2.boundingRect(contours[0])
    temp = result[y:(y + h), x:(x + w)]
    rect = cv2.minAreaRect(contours[0])
    height,width = temp.shape[:2]

    up = int((height - rect[1][0])/2)
    down = int(height - up)
    left = int((width - rect[1][1])/2)
    right =int(width - left)
    #第一个参数旋转中心，第二个参数旋转角度，第三个参数：缩放比例
    M = cv2.getRotationMatrix2D((width/2,height/2),(rect[2]+90),1)
    res = cv2.warpAffine(temp,M,(width,height))
    temp1 = res[up:down,left:right]
    gray1 = cv2.cvtColor(temp1,cv2.COLOR_BGR2GRAY)
    ret,thresh1 = cv2.threshold(gray1,190,255,cv2.THRESH_BINARY)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    eroded = cv2.erode(thresh1, kernel)
    
    file = "images_data/img/adjust.png"
    cv2.imwrite(file, eroded)
    #cv2.imwrite(file, thresh1)
    return file

# 水平垂直投影

In [ ]:
def Projection(file_name):
    img=cv2.imread(file_name)  #读取图片，装换为可运算的数组  
    GrayImage=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)   #将BGR图转为灰度图  
    ret,thresh=cv2.threshold(GrayImage,190,255,cv2.THRESH_BINARY)  #将图片进行二值化（190,255）之间的点均变为255（背景）  

    (height,width)=thresh.shape #返回高和宽    
    paint = np.zeros(img.shape, np.uint8)
    
    w = [0 for z in range(0, width)]  
    h = [0 for z in range(0, height)]  

    #记录每一列的波峰  
    for j in range(0,width): #遍历一列   
        for i in range(0,height):  #遍历一行  
            paint[i,j]=255
            if  thresh[i,j]==0:  
                w[j]+=1           
    for j in range(0,width):  #遍历每一列  
        for i in range((height-w[j]),height):  #从该列应该变黑的最顶部的点开始向最底部涂黑  
            paint[i,j]=0   #涂黑  
    cv2.imwrite("images_data/Vertical.png",paint)

    #记录每一行的波峰  
    for i in range(0,height):
        for j in range(0,width): #遍历一列   
            paint[i,j]=255  
            if thresh[i,j]==0:   
                h[i]+=1        
    for i in range(0,height):
        for j in range(0,width-(width-h[i])):
            paint[i,j]=0   #涂黑  
    cv2.imwrite("images_data/Horizontal.png",paint)
    return w,h,thresh
    # plt.imshow(thresh,cmap=plt.gray())  
    # plt.show()  
    # cv2.imshow('img',thresh)    
    # cv2.waitKey(0)    
    # cv2.destroyAllWindows() 

# 成绩表格识别计算

In [ ]:

x_axis = [] #列
y_axis = [] #行
Ver = []#垂直投影 len(w)
Hor = []#水平投影 len(h)
file_name = 'images_data/v1.png'
file = Adjust_img(file_name) #切割校正函数 
#Ver,Hor,thresh = Projection('images_data/v1.png')
Ver,Hor,thresh = Projection(file) #投影统计函数
for i in range(len(Hor)-2):
    if Hor[i]>850 and Hor[i+1]<850 and Hor[i+2]<850:
        x_axis.append(i)
for i in range(len(Ver)-2):
    if Ver[i]>180 and Ver[i+1]<180 and Ver[i+2]<180:
        y_axis.append(i)
#print(x_axis,y_axis)

mark = 0
for i in range(1,len(y_axis)-1):
    crop_img = thresh[x_axis[1]+3:x_axis[2]-3,y_axis[i]+3:y_axis[i+1]-3]
    cv2.imwrite("images_data/img/"+str(i)+".png", crop_img)
    ret,thresh1 = cv2.threshold(crop_img,190,255,cv2.THRESH_BINARY_INV)
    binary, contours, hierarchy = cv2.findContours( thresh1,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    #print(i,len(contours))
    a = 0
    x0 = 0
    for j in range(len(contours)):
        x1, y1, w1, h1 = cv2.boundingRect(contours[j])
        temp =  crop_img[y1-1:(y1 + h1+1), x1-1:(x1 + w1+1)]
        files = "images_data/img/" + str(i)+str(j) + ".png"
        cv2.imwrite(files, temp)
        imgs = Func.Thumbnail(Image.open(files).convert('1'),28)
        result = Func.Getpixels(imgs,0)
        prediction=tf.argmax(y_conv,1)
        predint=prediction.eval(feed_dict={x: [result],keep_prob: 1.0}, session=sess)#加[]可以将list变为矩阵
        #print(files,predint[0])
        if a == 0:
            a = predint[0]
        elif x1>x0:
            a = a*10+ predint[0]
        else :
            a += predint[0]*10
        x0 = x1
    print(a)
    mark += a
print(mark)

# 图片腐蚀膨胀

In [3]:

#运行如果报错，考虑是文件的名称含有中文字符的原因
file = 'images_data/ss1.png'
img = cv2.imread(file)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret,thresh = cv2.threshold(gray,190,255,cv2.THRESH_BINARY)
#ret,thresh = cv2.threshold(gray,190,255,cv2.THRESH_BINARY_INV) #反色读入
#cv2.imwrite("images_data/thresh.png", thresh)

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (4, 4))
#eroded = cv2.erode(thresh, kernel)
dilate = cv2.dilate(thresh,kernel)
#cv2.imwrite('images_data/sample.png', eroded)
cv2.imwrite('images_data/dilate.png', dilate)
kernel1 = cv2.getStructuringElement(cv2.MORPH_RECT, (8, 8))
eroded = cv2.erode( dilate, kernel1)
cv2.imwrite('images_data/eroded.png', eroded)


True

# 运算符数据采集

In [ ]:

def one_hot(labels):#转换为one_hot模式
    batch_size = tf.size(labels)
    labels = tf.expand_dims(labels, 1)
    indices = tf.expand_dims(tf.range(0, batch_size, 1), 1)
    concated = tf.concat([indices, labels],1)
    onehot_labels = tf.sparse_to_dense(concated, tf.stack([batch_size, 14]), 1.0, 0.0)
    return  onehot_labels

def get_files(file_dir):
    A0 = []
    label_0 = []
    for file in os.listdir(file_dir):
        A0.append(imageprepare(file_dir+file))
        name = file.split(sep='(')
        if name[0] == '+ ':
            label_0.append(10)
        elif name[0] == '- ':
            label_0.append(11)
        elif name[0] == '× ':
            label_0.append(12)
        elif name[0] == '÷ ':
            label_0.append(13)
        else:
            name1 = file.split(sep='.')
            label_0.append((int(name1[0])%10))
        #print(name,label_0[len(label_0)-1]) 
    return A0 ,label_0#返回两个list        
           
def imageprepare(file_name):
    img = Image.open(file_name).convert('L')#读取一副图像，转化为灰度图像
    tv = list(img.getdata()) #get pixel values
    #normalize pixels to 0 and 1. 0 is pure white, 1 is pure black.
    tva = [ (255-x)*1.0/255.0 for x in tv] 
    return tva

train_dir = 'images_data/+-x÷/'
#存放用来训练的图片的路径
image_list,label_list = get_files(train_dir)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
print(np.array(image_list).shape)  


# intro-扩大维度训练

In [ ]:

time_start=time.time();#time.time()为1970.1.1到当前时间的毫秒数 
 
from tensorflow.examples.tutorials.mnist import input_data  
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)  
def one_hot(labels):
    batch_size = tf.size(labels)
    labels = tf.expand_dims(labels, 1)
    indices = tf.expand_dims(tf.range(0, batch_size, 1), 1)
    concated = tf.concat([indices, labels],1)
    onehot_labels = tf.sparse_to_dense(concated, tf.stack([batch_size, 14]), 1.0, 0.0)
    return  onehot_labels

def random(size):
    import random
    batch_x = []
    batch_y = []
    slice = random.sample(range(0,160), size)  #从list中随机获取 个元素，作为一个片断返回  
    for i in range(size):
        batch_x.append(image_list[slice[i]])
        batch_y.append(label_list[slice[i]])
    batch_y = one_hot(batch_y)
    return batch_x ,batch_y

#define placeholder for inputs to netword
x = tf.placeholder("float", [None, 784]) 
y_ = tf.placeholder("float", [None,14])

W = tf.Variable(tf.zeros([784,14]),dtype = tf.float32,name = 'weights')
b = tf.Variable(tf.zeros([14]),dtype = tf.float32,name = 'biases')
y = tf.nn.softmax(tf.matmul(x,W) + b)
y_ = tf.placeholder("float", [None,14])
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
batch_num = 100

for i in range(501):
    batch_xs, batch_ys = mnist.train.next_batch(batch_num)
    img_data,img_label = random(10) #随机取多个样本
    train_x = tf.concat([batch_xs,np.array(img_data,dtype=np.float32)],0) 
    train_y =tf.concat([tf.concat([batch_ys,tf.zeros([batch_num,4])],1),img_label],0)  
    test_labels = tf.concat([mnist.test.labels,tf.zeros([10000,4])],1)
    
    sess.run(train_step, feed_dict={x: sess.run(train_x), y_: sess.run(train_y)})
    if i%50 == 0:
        correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        print ("step %d, training accuracy %g"%(i, sess.run(accuracy, feed_dict={x:mnist.test.images,y_:sess.run(test_labels)})))  
        
        
saver = tf.train.Saver()   
saver.save(sess, "./model/expand_intro/model.ckpt")
print(b.eval(session=sess))

time_end=time.time();#time.time()为1970.1.1到当前时间的毫秒数  
print ((time_end-time_start)/60,"mins")  

# expert-扩大维度训练

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data  
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)  

time_start=time.time();#time.time()为1970.1.1到当前时间的毫秒数  

sess = tf.InteractiveSession()  
def one_hot(labels):
    batch_size = tf.size(labels)
    labels = tf.expand_dims(labels, 1)
    indices = tf.expand_dims(tf.range(0, batch_size, 1), 1)
    concated = tf.concat([indices, labels],1)
    onehot_labels = tf.sparse_to_dense(concated, tf.stack([batch_size, 14]), 1.0, 0.0)
    return  onehot_labels

def random(size):
    import random
    batch_x = []
    batch_y = []
    slice = random.sample(range(0,210), size)  #从list中随机获取 个元素，作为一个片断返回  
    for i in range(size):
        batch_x.append(image_list[slice[i]])
        batch_y.append(label_list[slice[i]])
    batch_y = one_hot(batch_y)
    return batch_x ,batch_y

#定义一些函数：分配系数函数、分配偏置函数、卷积函数、pooling函数  
def weight_variable(shape):  
    initial = tf.truncated_normal(shape, stddev=0.1)#均值0标准方差0.1，剔除2倍标准方差之外的随机数据  
    return tf.Variable(initial)  
    
def bias_variable(shape):  
    initial = tf.constant(0.1, shape=shape)#统一值0.1  
    return tf.Variable(initial)  
  
def conv2d(x, W):  
  #待操作的数据x，模板W，tensor不同维度上的步长，强制与原tensor等大  
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')  
  
def max_pool_2x2(x):  
    #平面数据的pool模板2*2，平面数据滑动步长2*2（非重叠的pool）  
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')  
  
  
 #x是输入的图像，y_是对应的标签  
x = tf.placeholder(tf.float32, [None, 784])  
y_ = tf.placeholder(tf.float32, [None, 14])  
  
#第1层卷积层，Receptive Field 5＊5，单个batch生成32通道数据  
W_conv1 = weight_variable([5, 5, 1, 32])  
b_conv1 = bias_variable([32])  
  
#把图像向量还原成28＊28的图像  
x_image = tf.reshape(x, [-1,28,28,1])  
 
#第1个卷积层，使用了ReLU激活函数  
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)  
h_pool1 = max_pool_2x2(h_conv1)  
  
#第2层卷积层，Receptive Field 5＊5，单个batch 32通道生成64通道数据  
W_conv2 = weight_variable([5, 5, 32, 64])  
b_conv2 = bias_variable([64])  
  
#第2个卷积层  
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)  
h_pool2 = max_pool_2x2(h_conv2)  
  
#全链接层系数  
W_fc1 = weight_variable([7 * 7 * 64, 1024])  
b_fc1 = bias_variable([1024])  
  
#全链接层：把64通道数据展开方便全链接  
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])  
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)  
  
#全链层神经元使用dropout防止过拟合  
keep_prob = tf.placeholder("float")  
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)  
  
#softmax层系数  
W_fc2 = weight_variable([1024, 14])  
b_fc2 = bias_variable([14])  
  
#softmax层  
y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)  
  
#交叉熵和训练构型：AdamOptimizer适合这种求和的误差项  
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))  
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)  
  
#验证步骤的构型  
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))  
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))  
  
#初始化  
sess.run(tf.global_variables_initializer())  

#读取模型继续训练
saver = tf.train.Saver() 
saver.restore(sess, "./model/expand_expert/model.ckpt-9879")  

max_acc=0
t1 = 0
batch_num = 100
#开始训练  

saver = tf.train.Saver() 

for i in range(1001):  
    batch_xs, batch_ys = mnist.train.next_batch(batch_num)
    img_data,img_label = random(10) #随机取多个样本
    train_x = tf.concat([batch_xs,np.array(img_data,dtype=np.float32)],0) 
    train_y =tf.concat([tf.concat([batch_ys,tf.zeros([batch_num,4])],1),img_label],0)  
    test_labels = tf.concat([mnist.test.labels,tf.zeros([10000,4])],1)
      
    train_step.run(feed_dict={x:sess.run(train_x), y_:sess.run(train_y), keep_prob: 0.5}) 
    if i%10 == 0:  
        #验证的时候dropout=1.0，训练时=0.5  
        train_accuracy = accuracy.eval(feed_dict={
            x:sess.run(train_x), y_:sess.run(train_y), keep_prob: 1.0})  
        test_accuracy = accuracy.eval(feed_dict={
            x: mnist.test.images, y_: sess.run(test_labels), keep_prob: 1.0})
        time_end=(time.time()-time_start)/60;
        t = time_end - t1
        t1 = time_end
        print ("step %d, training accuracy %g,test accuracy %g,time %.01f mins ,%.01f "%(i, train_accuracy,test_accuracy,time_end,t))
        if  test_accuracy>max_acc:
            max_acc=test_accuracy
            saver.save(sess, "./model/expand_expert/model.ckpt",global_step= int( max_acc*10000) ) 
 
time_end=time.time();
print ((time_end-time_start)/60,"mins")  

# restore intro_model

In [ ]:
import tensorflow as tf
import numpy as np

# restore variables
# redefine the same shape and same type for your variables
W = tf.Variable(np.arange((784*14)).reshape((784, 14)), dtype=tf.float32, name="weights")
bias = tf.Variable(np.arange(14), dtype=tf.float32, name="biases")
sess = tf.Sessio÷n()
#sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

saver.restore(sess, "./expand_intro/model.ckpt")  
print(bias.eval(session=sess))
print("weights:", sess.run(W))
print("biases:", sess.run(bias))

# restore expert_model

In [ ]:

  
#定义一些函数：分配系数函数、分配偏置函数、卷积函数、pooling函数  
def weight_variable(shape):  
    initial = tf.truncated_normal(shape, stddev=0.1)#均值0标准方差0.1，剔除2倍标准方差之外的随机数据  
    return tf.Variable(initial)  
def bias_variable(shape):  
    initial = tf.constant(0.1, shape=shape)#统一值0.1  
    return tf.Variable(initial)   
def conv2d(x, W):  
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')   
def max_pool_2x2(x):  
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')  

x = tf.placeholder(tf.float32, [None, 784])  
y_ = tf.placeholder(tf.float32, [None, 14])  
W_conv1 = weight_variable([5, 5, 1, 32])  
b_conv1 = bias_variable([32])  
x_image = tf.reshape(x, [-1,28,28,1])  
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)  
h_pool1 = max_pool_2x2(h_conv1)  
W_conv2 = weight_variable([5, 5, 32, 64])  
b_conv2 = bias_variable([64])  
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)  
h_pool2 = max_pool_2x2(h_conv2)  
W_fc1 = weight_variable([7 * 7 * 64, 1024])  
b_fc1 = bias_variable([1024])  
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])  
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)  
keep_prob = tf.placeholder("float")  
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)  
W_fc2 = weight_variable([1024, 14])  
b_fc2 = bias_variable([14])  
y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)  
#恢复模型
sess = tf.Session()
sess.run(tf.global_variables_initializer())  
saver = tf.train.Saver() 
saver.restore(sess, "./model/expand_expert/model.ckpt-9878")  


# intro-测试自己的图片

In [ ]:

file_dir ='img/' #导入自己的图片地址
for file in os.listdir(file_dir):
    result = Func.imageprepare(file_dir+file)
    z = tf.placeholder("float", [1, 784])  #28*28
    y1 = tf.nn.softmax(tf.matmul(z,W) + b)
    prediction=tf.argmax(y1,1)
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    predint=prediction.eval(feed_dict={z: [result]}, session=sess)
    print('recognize result:',predint[0])

# expert-测试自己的图片

In [ ]:

file_dir ='images_data/img/' #导入自己的图片地址
for file in os.listdir(file_dir):
    img = Func.Thumbnail(Image.open(file_dir+file).convert('1'),28)
    result = Func.Getpixels(img,0)
    prediction=tf.argmax(y_conv,1)
    predint=prediction.eval(feed_dict={x: [result],keep_prob: 1.0}, session=sess)#加[]可以将list变为矩阵
    print(file,predint[0])

# CNN读取模型继续训练

In [ ]:

def get_files(file_dir):
    A0 = []
    label_0 = []
    for file in os.listdir(file_dir):
        A0.append(Func.Getpixels(file_dir+file,1))
        name = file.split(sep='(')
        if name[0] == '+ ':
            label_0.append(10)
        elif name[0] == '- ':
            label_0.append(11)
        elif name[0] == '× ':
            label_0.append(12)
        elif name[0] == '÷ ':
            label_0.append(13)
        else:
            name1 = file.split(sep='.')
            label_0.append((int(name1[0])%10))
        #print(name,label_0[len(label_0)-1]) 
    return A0 ,label_0#返回两个list        
           
train_dir = 'images_data/+-x÷/'
#存放用来训练的图片的路径
image_list,label_list = get_files(train_dir)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print(np.array(image_list).shape)  


In [ ]:


from tensorflow.examples.tutorials.mnist import input_data  
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)  

time_start=time.time();#time.time()为1970.1.1到当前时间的毫秒数  

sess = tf.InteractiveSession()  

#定义一些函数：分配系数函数、分配偏置函数、卷积函数、pooling函数  
def weight_variable(shape):  
    initial = tf.truncated_normal(shape, stddev=0.1)#均值0标准方差0.1，剔除2倍标准方差之外的随机数据  
    return tf.Variable(initial)  
    
def bias_variable(shape):  
    initial = tf.constant(0.1, shape=shape)#统一值0.1  
    return tf.Variable(initial)  
  
def conv2d(x, W):  
  #待操作的数据x，模板W，tensor不同维度上的步长，强制与原tensor等大  
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')  
  
def max_pool_2x2(x):  
    #平面数据的pool模板2*2，平面数据滑动步长2*2（非重叠的pool）  
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')  
  
  
 #x是输入的图像，y_是对应的标签  
x = tf.placeholder(tf.float32, [None, 784])  
y_ = tf.placeholder(tf.float32, [None, 14])  
  
#第1层卷积层，Receptive Field 5＊5，单个batch生成32通道数据  
W_conv1 = weight_variable([5, 5, 1, 32])  
b_conv1 = bias_variable([32])  
  
#把图像向量还原成28＊28的图像  
x_image = tf.reshape(x, [-1,28,28,1])  
 
#第1个卷积层，使用了ReLU激活函数  
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)  
h_pool1 = max_pool_2x2(h_conv1)  
  
#第2层卷积层，Receptive Field 5＊5，单个batch 32通道生成64通道数据  
W_conv2 = weight_variable([5, 5, 32, 64])  
b_conv2 = bias_variable([64])  
  
#第2个卷积层  
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)  
h_pool2 = max_pool_2x2(h_conv2)  
  
#全链接层系数  
W_fc1 = weight_variable([7 * 7 * 64, 1024])  
b_fc1 = bias_variable([1024])  
  
#全链接层：把64通道数据展开方便全链接  
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])  
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)  
  
#全链层神经元使用dropout防止过拟合  
keep_prob = tf.placeholder("float")  
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)  
  
#softmax层系数  
W_fc2 = weight_variable([1024, 14])  
b_fc2 = bias_variable([14])  
  
#softmax层  
y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)  
  
#交叉熵和训练构型：AdamOptimizer适合这种求和的误差项  
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))  
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)  
  
#验证步骤的构型  
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))  
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))  
  
#初始化  
sess.run(tf.global_variables_initializer())  

#读取模型继续训练
saver = tf.train.Saver() 
#saver.restore(sess, "./model/expand_expert/model.ckpt-9892")  

max_acc=0
t1 = 0
batch_num = 50
#开始训练  
for i in range(501):  
    batch_xs, batch_ys = mnist.train.next_batch(batch_num)
    img_data,img_label = Func.random(15,len(image_list),image_list,label_list) #随机取多个样本
    train_x = tf.concat([batch_xs,np.array(img_data,dtype=np.float32)],0) 
    train_y =tf.concat([tf.concat([batch_ys,tf.zeros([batch_num,4])],1),img_label],0)  
    test_labels = tf.concat([mnist.test.labels,tf.zeros([10000,4])],1)
      
    train_step.run(feed_dict={x:sess.run(train_x), y_:sess.run(train_y), keep_prob: 0.5}) 
    if i%10 == 0:  
        #验证的时候dropout=1.0，训练时=0.5  
        train_accuracy = accuracy.eval(feed_dict={
            x:sess.run(train_x), y_:sess.run(train_y), keep_prob: 1.0})  
        test_accuracy = accuracy.eval(feed_dict={
            x: mnist.test.images, y_: sess.run(test_labels), keep_prob: 1.0})
        time_end=(time.time()-time_start)/60;
        t = time_end - t1
        t1 = time_end
        print ("step %d, training accuracy %g,test accuracy %g,time %.01f mins ,%.01f "%(i, train_accuracy,test_accuracy,time_end,t))
        if  test_accuracy>max_acc and i>50:
            max_acc=test_accuracy
            #saver.save(sess, "./model/expand_expert/model.ckpt",global_step= int( max_acc*10000) ) 

            
time_end=time.time();
print ((time_end-time_start)/60,"mins")  

# TensorBoard 

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data  
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)  

time_start=time.time()
sess = tf.InteractiveSession()  
  
#定义一些函数：分配系数函数、分配偏置函数、卷积函数、pooling函数  
def weight_variable(shape):  
    initial = tf.truncated_normal(shape, stddev=0.1)#均值0标准方差0.1，剔除2倍标准方差之外的随机数据  
    return tf.Variable(initial)  
    
def bias_variable(shape):  
    initial = tf.constant(0.1, shape=shape)#统一值0.1  
    return tf.Variable(initial)  
  
def conv2d(x, W):  
  #待操作的数据x，模板W，tensor不同维度上的步长，强制与原tensor等大  
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')  
  
def max_pool_2x2(x):  
    #平面数据的pool模板2*2，平面数据滑动步长2*2（非重叠的pool）  
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')  
  
with tf.name_scope('inputs'):
    #x是输入的图像，y_是对应的标签  
    x = tf.placeholder(tf.float32, [None, 784],name='x_input')  
    y_ = tf.placeholder(tf.float32, [None, 10], name='y_input')  
    
#把图像向量还原成28＊28的图像  
with tf.name_scope('input_reshape1'):  
    x_image = tf.reshape(x, [-1,28,28,1]) 
    
#第1层卷积层，Receptive Field 5＊5，单个batch生成32通道数据 使用了ReLU激活函数  
with tf.name_scope('conv1'):  
    W_conv1 = weight_variable([5, 5, 1, 32])  
    b_conv1 = bias_variable([32])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)  
with tf.name_scope('pool1'):  
    h_pool1 = max_pool_2x2(h_conv1)  
    
#第2层卷积池化层，Receptive Field 5＊5，单个batch 32通道生成64通道数据  
with tf.name_scope('conv2'):  
    W_conv2 = weight_variable([5, 5, 32, 64])  
    b_conv2 = bias_variable([64]) 
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)  
with tf.name_scope('pool2'):  
    h_pool2 = max_pool_2x2(h_conv2)  

  

  
#全链接层：把64通道数据展开方便全链接    
with tf.name_scope('full_connect'):
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    W_fc1 = weight_variable([7 * 7 * 64, 1024])  
    b_fc1 = bias_variable([1024])  
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)  

#全链层神经元使用dropout防止过拟合  
with tf.name_scope('dropout'): 
    keep_prob = tf.placeholder("float")  
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)  

#softmax层系数 
with tf.name_scope('softmax'):
    W_fc2 = weight_variable([1024, 10])  
    b_fc2 = bias_variable([10]) 
    y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)  
tf.summary.histogram( 'outputs', y_conv)
with tf.name_scope('result'):
    #交叉熵和训练构型：AdamOptimizer适合这种求和的误差项  
    cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))  
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)  
  
    #验证步骤的构型  
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))  
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    
tf.summary.scalar('cross_entropy', cross_entropy)
tf.summary.scalar("accuracy", accuracy)
#初始化  
sess.run(tf.global_variables_initializer())  
  
#清空文件夹
file_dir ='logs/'
for file in os.listdir(file_dir):
    os.remove(file_dir+file)
    
    
merged =  tf.summary.merge_all()
#summary_op = tf.summary.merge([cost_summary, accuracy_summary])
writer = tf.summary.FileWriter("logs/", sess.graph)
#开始训练  
for i in range(1000):  
    batch = mnist.train.next_batch(50)  
    if i%50 == 0:  
        #验证的时候dropout=1.0，训练时=0.5  
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y_: batch[1], keep_prob: 1.0})  
        print ("step %d, training accuracy %g"%(i, train_accuracy))  
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})  
    #result = sess.run(merged,feed_dict={xs: x_data, ys: y_data})
        #writer.add_summary(result, i)
    summary_str = sess.run(merged, feed_dict={x:batch[0], y_:batch[1], keep_prob:(1.0)})
    writer.add_summary(summary_str, i)

#验证最终的准确率  
print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}) )  

#saver = tf.train.Saver()   
#saver.save(sess, "./model/expert/model.ckpt")  

writer.close() ##程序运行结束后关闭文件并刷新到硬盘
time_end=time.time();#time.time()为1970.1.1到当前时间的毫秒数  
print ((time_end-time_start)/60,"mins")  

In [ ]:
tensorboard --logdir=D://Jupyter//logs

# Formula Recognition

In [ ]:
头文件部分
from PIL import Image, ImageFilter
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2
import Func
import time
import os
%matplotlib inline

读取模型部分
#定义一些函数：分配系数函数、分配偏置函数、卷积函数、pooling函数  
def weight_variable(shape):  
    initial = tf.truncated_normal(shape, stddev=0.1)#均值0标准方差0.1，剔除2倍标准方差之外的随机数据  
    return tf.Variable(initial)  
def bias_variable(shape):  
    initial = tf.constant(0.1, shape=shape)#统一值0.1  
    return tf.Variable(initial)   
def conv2d(x, W):  
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')   
def max_pool_2x2(x):  
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')  

x = tf.placeholder(tf.float32, [None, 784])  
y_ = tf.placeholder(tf.float32, [None, 14])  
W_conv1 = weight_variable([5, 5, 1, 32])  
b_conv1 = bias_variable([32])  
x_image = tf.reshape(x, [-1,28,28,1])  
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)  
h_pool1 = max_pool_2x2(h_conv1)  
W_conv2 = weight_variable([5, 5, 32, 64])  
b_conv2 = bias_variable([64])  
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)  
h_pool2 = max_pool_2x2(h_conv2)  
W_fc1 = weight_variable([7 * 7 * 64, 1024])  
b_fc1 = bias_variable([1024])  
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])  
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)  
keep_prob = tf.placeholder("float")  
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)  
W_fc2 = weight_variable([1024, 14])  
b_fc2 = bias_variable([14])  
y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)  
#恢复模型
sess = tf.Session()
sess.run(tf.global_variables_initializer())  
saver = tf.train.Saver() 
saver.restore(sess, "./model/expand_expert/model.ckpt-9892")  


对文件夹内的每个图片进行识别的部分

#清空文件夹
file_dir ='images_data/img/'
for file in os.listdir(file_dir):
    os.remove(file_dir+file)
    

file_name='images_data/formula/formula2.png'
img = cv2.imread(file_name)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#120  124 150 3
ret,thresh = cv2.threshold(gray,120,255,cv2.THRESH_BINARY)
#ret,thresh = cv2.threshold(gray,190,255,cv2.THRESH_BINARY_INV) #反色读入
#cv2.imwrite("images_data/img/thresh.png", thresh)

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (4,4))
dilate = cv2.dilate(thresh,kernel)
eroded = cv2.erode(dilate, kernel)
#cv2.imwrite('images_data/img/dilate.png', dilate)
file1 = 'images_data/img/eroded.png'
cv2.imwrite(file1, eroded)
eroded = cv2.imread(file1)


plt.imshow(img)
plt.title("Original picture")
plt.xticks([])
plt.yticks([])
plt.show()
plt.imshow(eroded)
plt.title("Binary picture")
plt.xticks([])
plt.yticks([])
plt.show()



im = Image.open(file1).convert('1')
w,h = im.size
edge = [0,0,0,0] #上下左右
lock = [0,0]
cnt = [0,0]
num = []
for i in range(h): #每一行
    if Func.Judgezero(im,i,0)==1 and lock[0]==0:
        edge[0],lock[0] = i,1
    if Func.Judgezero(im,i,0)==0 and lock[0]==1:
        edge[1],lock[0] = i,0
        cnt[0] += 1
        #print(edge[0],edge[1])
        img0 = im.crop((0,edge[0],w,edge[1]))
        img0.save(file_dir+"img"+str(cnt[0])+".png") 
        cnt[1]= 0
        for j in range(w):# 每一列
            if Func.Judgezero(img0,j,1)==1 and lock[1]==0:
                edge[2],lock[1] = j,1
            if Func.Judgezero(img0,j,1)==0 and lock[1]==1:
                edge[3],lock[1] = j,0
                cnt[1] += 1
                #print(edge[2],edge[3])
                img1 = img0.crop((edge[2],0,edge[3],edge[1]-edge[0]))
                #print((img1.crop(Func.JudgeEdge(img1)).size))
                img2 = Func.Thumbnail(img1.crop(Func.JudgeEdge(img1)),28)
                img2.save(file_dir+"img"+str(cnt[0])+'%02d' %cnt[1]+".png")    
        
        num.append(cnt[1])
print(num)


for i in range(len(num)):
    ans = []
    for j in range(num[i]):
        img = Func.Thumbnail(Image.open(file_dir+"img"+str(i+1)+'%02d' %(j+1)+".png").convert('1'),28)
        plt.subplot(1,num[i],j+1)
        plt.imshow(img)
        plt.xticks([])
        plt.yticks([])
        result = Func.Getpixels(img,0)
        prediction=tf.argmax(y_conv,1)
        predint=prediction.eval(feed_dict={x: [result],keep_prob: 1.0}, session=sess)#加[]可以将list变为矩阵
        if predint[0] == 10:
            ans.append("+")
        elif predint[0] == 11:
            ans.append("-")
        elif predint[0] == 12:
            ans.append("*")
        elif predint[0] == 13:
            ans.append("/")
        else :
            ans.append(predint[0])
    plt.show()
    answer = "".join('%s' %id for id in ans)
    #print('list:',ans) 
    #print('string:',answer)
    print("answer:",answer,"=",eval(answer))


# HNR

In [ ]:
file_dir ='images_data/num/' #导入自己的图片地址
for file in os.listdir(file_dir):
    ans = []
    img = Func.Thumbnail(Image.open(file_dir+file).convert('1'),28)
    #plt.imshow(img)
    #plt.xticks([])
    #plt.yticks([])
    #plt.show()
    result = Func.Getpixels(img,0)
    prediction=tf.argmax(y_conv,1)
    predint=prediction.eval(feed_dict={x: [result],keep_prob: 1.0}, session=sess)#加[]可以将list变为矩阵
    if predint[0] == 10:
        ans.append("＋")
    elif predint[0] == 11:
        ans.append("－")
    elif predint[0] == 12:
        ans.append("×")
    elif predint[0] == 13:
        ans.append("÷")
    else :
        ans.append(predint[0])
    #file=file.PadLeft(20, " ");
    print("file_name:",file,"   result:",ans[0])

# Score  Recognition

In [3]:
def Adjust_img(file_name):
    img = cv2.imread(file_name)
    plt.imshow(img)
    plt.title("Original picture")
    plt.xticks([])
    plt.yticks([])
    plt.show()
    result = img.copy()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    #ret,thresh = cv2.threshold(gray,190,255,cv2.THRESH_BINARY)
    ret,thresh = cv2.threshold(gray,150,255,cv2.THRESH_BINARY_INV) #反色读入

    #binary, contours, hierarchy = cv2.findContours(eroded,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    binary, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

    x, y, w, h = cv2.boundingRect(contours[0])
    temp = result[y:(y + h), x:(x + w)]
    rect = cv2.minAreaRect(contours[0])
    height,width = temp.shape[:2]

    up = int((height - rect[1][0])/2)
    down = int(height - up)
    left = int((width - rect[1][1])/2)
    right =int(width - left)
    #第一个参数旋转中心，第二个参数旋转角度，第三个参数：缩放比例
    M = cv2.getRotationMatrix2D((width/2,height/2),(rect[2]+90),1)
    res = cv2.warpAffine(temp,M,(width,height))
    temp1 = res[up:down,left:right]
    gray1 = cv2.cvtColor(temp1,cv2.COLOR_BGR2GRAY)
    ret,thresh1 = cv2.threshold(gray1,190,255,cv2.THRESH_BINARY)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    eroded = cv2.erode(thresh1, kernel)
    
    file = "images_data/img/adjust.png"
    cv2.imwrite(file, eroded)
    #plt.tilte('Title in a custom color',color='#123456'）  
    plt.imshow(eroded)
    plt.title("Correction picture")
    plt.xticks([])
    plt.yticks([])
    plt.show()
    #cv2.imwrite(file, thresh1)
    return file

In [4]:
def Projection(file_name):
    img=cv2.imread(file_name)  #读取图片，装换为可运算的数组  
    GrayImage=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)   #将BGR图转为灰度图  
    ret,thresh=cv2.threshold(GrayImage,190,255,cv2.THRESH_BINARY)  #将图片进行二值化（190,255）之间的点均变为255（背景）  
    cv2.imwrite("images_data/img/adjust.png", thresh)
    (height,width)=thresh.shape #返回高和宽    
    paint = np.zeros(img.shape, np.uint8)
    
    w = [0 for z in range(0, width)]  
    h = [0 for z in range(0, height)]  

    #记录每一列的波峰  
    for j in range(0,width): #遍历一列   
        for i in range(0,height):  #遍历一行  
            paint[i,j]=255
            if  thresh[i,j]==0:  
                w[j]+=1           
    for j in range(0,width):  #遍历每一列  
        for i in range((height-w[j]),height):  #从该列应该变黑的最顶部的点开始向最底部涂黑  
            paint[i,j]=0   #涂黑  
    cv2.imwrite("images_data/img/Vertical.png",paint)
    plt.imshow(paint)
    plt.title("Vertical projection")
    plt.xticks([])
    plt.yticks([])
    plt.show()

    #记录每一行的波峰  
    for i in range(0,height):
        for j in range(0,width): #遍历一列   
            paint[i,j]=255  
            if thresh[i,j]==0:   
                h[i]+=1        
    for i in range(0,height):
        for j in range(0,width-(width-h[i])):
            paint[i,j]=0   #涂黑  
    cv2.imwrite("images_data/img/Horizontal.png",paint)
    plt.imshow(paint)  
    plt.title("Horizontal projection")
    plt.xticks([])
    plt.yticks([])
    plt.show()  
    return w,h,thresh

# Score  Recognition2（非投影方式）

In [ ]:
#清空文件夹
file_dir ='images_data/img/'
for file in os.listdir(file_dir):
    os.remove(file_dir+file)
    

file_name='images_data/score/score4.png'
img = cv2.imread(file_name)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret,thresh = cv2.threshold(gray,180,255,cv2.THRESH_BINARY)
#ret,thresh = cv2.threshold(gray,190,255,cv2.THRESH_BINARY_INV) #反色读入
#cv2.imwrite("images_data/img/thresh.png", thresh)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
dilate = cv2.dilate(thresh,kernel)
eroded = cv2.erode(dilate, kernel)
#cv2.imwrite('images_data/img/dilate.png', dilate)
file1 = 'images_data/img/eroded.png'
cv2.imwrite(file1, eroded)
eroded = cv2.imread(file1)

#plt.subplot(1,2,1)
plt.imshow(img)
plt.title("Original picture")
plt.xticks([])
plt.yticks([])
plt.show()
#plt.subplot(1,2,2)
plt.imshow(eroded)
plt.title("Binary picture")
plt.xticks([])
plt.yticks([])
plt.show()



im = Image.open(file1).convert('1')
w,h = im.size

img= im.crop(Func.JudgeEdge(im))
img.save('sample.png')        
w,h = img.size
A = []
mark = 0



img1 = cv2.imread('sample.png')
gray = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)

start = [0.155,0.26,0.365,0.47,0.578,0.685,0.79]
end = [0.245,0.35,0.46,0.57,0.675,0.78,0.87] 
for i in range(1,8):
    crop_img = gray[int(0.36*h):int(0.625*h),int(start[i-1]*w):int(end[i-1]*w)]
    plt.subplot(1,7,i)
    plt.imshow(crop_img)  
    #plt.axis('off')
    plt.xticks([])
    plt.yticks([])
    cv2.imwrite("images_data/img/"+str(i)+".png", crop_img)
    
    ret,thresh1 = cv2.threshold(crop_img,140,255,cv2.THRESH_BINARY_INV)
    
    binary, contours, hierarchy = cv2.findContours(thresh1,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    #print(i,len(contours))
    a = 0
    x0 = 0
    for j in range(len(contours)):
        x1, y1, w1, h1 = cv2.boundingRect(contours[j])
        temp =  crop_img[y1-1:(y1 + h1+1), x1-1:(x1 + w1+1)]
        files = "images_data/img/" + str(i)+str(j) + ".png"
        cv2.imwrite(files, temp)
        imgs = Func.Thumbnail(Image.open(files).convert('1'),28)
        result = Func.Getpixels(imgs,0)
        prediction=tf.argmax(y_conv,1)
        predint=prediction.eval(feed_dict={x: [result],keep_prob: 1.0}, session=sess)#加[]可以将list变为矩阵
        #print(files,predint[0])
        if a == 0:
            a = predint[0]
        elif x1>x0:
            a = a*10+ predint[0]
        else :
            a += predint[0]*10
        x0 = x1
    A.append(a)
    #print(a)
    mark += a
  
plt.xticks([])
plt.yticks([])
plt.show() 
print(A)
print("ans =" ,mark)

# 笔记

In [ ]:
重启内核会使运算加快
读取模型需要重启内核
截取图片要把图片放到中间，四周留出一定的空白 识别率会高一些

In [ ]:
c=tf.concat([W,batch_ys],1) #连接两个tensor
print('c.shape:',c.shape)#显示tensor格式
tf.reshape(100,14)#格式变化，但是数据总量不变
print(c.eval(session=sess))#显示tensor的数据
print(y1.eval(feed_dict={z: [result]}, session=sess))

plt.imshow(im)  
plt.title("Original picture")
plt.xticks([])
plt.yticks([])
plt.show()
plt.axis('off')

In [ ]:
import cv2  
img = cv2.imread( file_name)#读取图片
img.shape #图片的size
img = np.zeros([64,64], np.uint8)  #cv2新建图片
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #转换为灰度图
ret,thresh = cv2.threshold(img,190,255,cv2.THRESH_BINARY_INV)#二值化
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 10))#腐蚀内核
eroded = cv2.erode(gray, kernel)#腐蚀图片
image = img.copy()  #复制图片
cv2.imwrite('images/thresh2.png', thresh)#保存图片
img=cv2.resize(res,(width,height),interpolation=cv2.INTER_CUBIC)#改变图片尺寸大小

In [ ]:
import numpy as np
a=[[1,2],[2,3],[3,4]]
b=[[1,2,3], [2,3,4], [3,4,5]]
print(np.hstack((a, b)))#数组组合

d = np.array([[1, 2], [5, 6], [9, 10]], dtype=np.float) #何强制生成一个 float 类型的数组

In [ ]:
import tensorflow as tf #one_hot数据格式转化
sess = tf.Session()
labels = [12,13,5,7,9]
batch_size = tf.size(labels)
labels = tf.expand_dims(labels, 1)
indices = tf.expand_dims(tf.range(0, batch_size, 1), 1)
concated = tf.concat([indices, labels],1)
onehot_labels = tf.sparse_to_dense(concated, tf.stack([batch_size, 14]), 1.0, 0.0)
sess.run(onehot_labels )

In [ ]:
'''
读取模型要注意先重启内核！！
注意保存和读取程序不要在一个文档里运行，容易报错，最好在两个里单独运行，
而且读取出错误可以重启kernel来再次运行，运行中发现连续两次运行读取程序，
第一次运行结果是正常保存的值，而第二次运行的就是初始化的零值，
第三次又是正常值，读取次数多了就会报错了
恢复模型后在print（b）过程中不要初始化，否则会导致sess运行的是初始化的值，即会出现全为零的情况
'''
import tensorflow as tf  
import numpy as np
#Save to file
#remember to define the same dtype and shape when restore
W = tf.Variable([[9,4,3,5],[5,4,7,8]],dtype = tf.float32,name = 'weights')
b = tf.Variable([[3,6,9]],dtype = tf.float32,name = 'biases')

saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    save_path = saver.save(sess,"my_net/save_net.ckpt")
    print("Save to path:",save_path)
    
################################################
import tensorflow as tf
import numpy as np

# restore variables
# redefine the same shape and same type for your variables
W = tf.Variable(np.arange(8).reshape((2, 4)), dtype=tf.float32, name="weights")
b = tf.Variable(np.arange(3).reshape((1, 3)), dtype=tf.float32, name="biases")

# not need init step
#saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, "my_net/save_net.ckpt")
    print("weights:", sess.run(W))
    print("biases:", sess.run(b))


# Func.py函数（需要保存成py文件再运行）

In [ ]:
# coding: utf-8

# In[ ]:
from PIL import Image, ImageFilter
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2
import Func
import time
import os

#读取图像转化为标准尺寸，返回图片的像素列表
def Getpixels(im,flag): #flag = 1 为地址
    if flag==1:
        im = Image.open(im).convert('L')#读取一副图像，转化为灰度图像
    tv = list(im.getdata()) #get pixel values
    tva = [ (255-x)*1.0/255.0 for x in tv] 
    return tva

#判断某一列全为零 返回值为0/1 全零为0 非全零为1
def Judgezero(img,num,flag):#判断某一行、列是否全为零 全零为0  flag 0 行判断 1 列判断
    pixs = np.asarray(img)
    if flag == 0: #判断一行
        result = 1 if list(pixs[num,:]==0).count(1) > 3 else 0
    else : #判断一列
        result = 1 if list(pixs[:,num]==0).count(1) > 3 else 0
    return result

def JudgeEdge(img): 
    pixs = np.asarray(img)
    size = [0,0,0,0]
    #寻找上下边界 
    for i in range(len(pixs)):
        if size[1]==0:
            if Judgezero(pixs,i,0) == 1:
                size[1] = i
        if size[3]==0:
            if Judgezero(pixs,len(pixs)-i-1,0) == 1:
                size[3]=len(pixs)-i-1 
        if size[1] != 0 and size[3] != 0:
            break
    #寻找左右边界
    for i in range(len(pixs[0])):
        if size[0]==0:
            if Judgezero(pixs,i,1) == 1:
                size[0] = i
        if size[2]==0:
            if Judgezero(pixs,len(pixs[0])-i-1,1) == 1:
                size[2]=len(pixs[0])-i-1 
        if size[0] != 0 and size[2] != 0:
            break
    return size

def Thumbnail(img,leng):
    
    width = float(img.size[0]) 
    height = float(img.size[1]) 
    newImage = Image.new('1', (leng, leng), (255)) # creates white canvas of 28x28 pixels 
    if width > height: # check which dimension is bigger 
        # Width is bigger. Width becomes 20 pixels. 
        nheight = int(round(((leng-4) / width * height), 0)) # resize height according to ratio width 
        if (nheight <5): # rare case but minimum is 1 pixel 
            nheigth = 5
            # resize and sharpen
        img = img.resize(((leng-4), nheight), Image.ANTIALIAS).filter(ImageFilter.SHARPEN) 
        wtop = int(round(((leng - nheight) / 2), 0)) # caculate horizontal pozition 
        newImage.paste(img, (2, wtop)) # paste resized image on white canvas 
    else: 
        # Height is bigger. Heigth becomes 20 pixels. 
        nwidth = int(round(((leng-4) / height * width), 0)) # resize width according to ratio height 
        if (nwidth <5): # rare case but minimum is 1 pixel 
            nwidth = 5
            # resize and sharpen 
        img = img.resize((nwidth, leng-4), Image.ANTIALIAS).filter(ImageFilter.SHARPEN) 
        wleft = int(round(((leng - nwidth) / 2), 0)) # caculate vertical pozition 
        newImage.paste(img, (wleft, 2)) # paste resized image on white canvas 
    return newImage

def one_hot(labels):#转换为one_hot模式
    batch_size = tf.size(labels)
    labels = tf.expand_dims(labels, 1)
    indices = tf.expand_dims(tf.range(0, batch_size, 1), 1)
    concated = tf.concat([indices, labels],1)
    onehot_labels = tf.sparse_to_dense(concated, tf.stack([batch_size, 14]), 1.0, 0.0)
    return  onehot_labels

def random(size,rge,image_list,label_list):
    import random
    batch_x = []
    batch_y = []
    slice = random.sample(range(0,rge), size)  #从list中随机获取 个元素，作为一个片断返回  
    for i in range(size):
        batch_x.append(image_list[slice[i]])
        batch_y.append(label_list[slice[i]])
    #batch_y = Func.one_hot(batch_y)
    batch_y = one_hot(batch_y)
    return batch_x ,batch_y